In [62]:
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [63]:
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"

DATA_FILE = DATA_DIR / "motley-fool-data.pkl"

DATA_FILE

WindowsPath('C:/Users/ssmyt/earnings-call-nlp/data/motley-fool-data.pkl')

In [64]:
with open(DATA_FILE, "rb") as f:
    data = pickle.load(f)
type(data)

pandas.core.frame.DataFrame

In [65]:
df = data.copy()
df.shape

(18755, 5)

In [66]:
df.head()

,date,exchange,q,ticker,transcript
0,"Aug 27, 2020, 9:00 p.m. ET",NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel..."
1,"Jul 30, 2020, 4:30 p.m. ET",NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...
2,"Oct 23, 2019, 5:00 p.m. ET",NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...
3,"Nov 6, 2019, 12:00 p.m. ET",NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon..."
4,"Aug 7, 2019, 8:30 a.m. ET",NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18755 entries, 0 to 18754
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        18755 non-null  object
 1   exchange    18755 non-null  object
 2   q           18755 non-null  object
 3   ticker      18755 non-null  object
 4   transcript  18755 non-null  object
dtypes: object(5)
memory usage: 732.7+ KB


In [68]:
#Check to see how dates are formatted 
df["date"].head()

0    Aug 27, 2020, 9:00 p.m. ET
1    Jul 30, 2020, 4:30 p.m. ET
2    Oct 23, 2019, 5:00 p.m. ET
3    Nov 6, 2019, 12:00 p.m. ET
4     Aug 7, 2019, 8:30 a.m. ET
Name: date, dtype: object

In [71]:
df.iloc[28]['date']

['Brunswick (BC 0.66%) Q4 2018 ', 'Jan. 31, 2019 11:00 a.m. ET']

In [43]:
df.loc[~df["date"].str.contains("ET", na=False), "date"].head()

28     [Brunswick (BC 0.66%) Q4 2018 , Jan. 31, 2019 ...
125    [Blucora (BCOR 1.57%) Q4 2018 , Feb. 14, 2019 ...
154    [Hoegh LNG Partners LP  (HMLP)Q4 2018 , Feb. 2...
161    [AECOM Technology (ACM 0.87%) Q1 2019 , Feb. 5...
235    [GATX Corp  (GATX 0.43%)Q4 2018 , Jan. 22, 201...
Name: date, dtype: object

In [74]:
date_list = df.iloc[28]['date']
date_str = next(item for item in date_list if 'ET' in item)
print(date_str)

Jan. 31, 2019 11:00 a.m. ET


In [80]:
date_str = re.sub(r"\sET$", "", date_str)  # remove ' ET'
date_str = date_str.replace("a.m.", "AM").replace("p.m.", "PM")
date_str
call_datetime = pd.to_datetime(date_str)
print(call_datetime)

2019-01-31 11:00:00


In [81]:
import pandas as pd
import re
import ast

def parse_clean_date(x):
    """
    Parse the 'date' column from Motley Fool dataset, handling:
    - Strings
    - Lists
    - Stringified lists
    - Dots in month abbreviations
    - ET timezone labels
    - a.m./p.m. normalization
    """
    
    # Step 1: Extract actual date string
    date_str = None
    
    # If x is a list
    if isinstance(x, list):
        for item in x:
            if isinstance(item, str) and "ET" in item:
                date_str = item
                break
    
    # If x is a string
    elif isinstance(x, str):
        # Stringified list?
        if x.startswith("[") and x.endswith("]"):
            try:
                items = ast.literal_eval(x)
                for item in items:
                    if isinstance(item, str) and "ET" in item:
                        date_str = item
                        break
            except:
                return pd.NaT
        else:
            date_str = x
    else:
        return pd.NaT

    if date_str is None:
        return pd.NaT

    # Step 2: Clean string
    date_str = re.sub(r"\sET$", "", date_str)        # remove ET
    date_str = date_str.replace("a.m.", "AM").replace("p.m.", "PM")  # normalize AM/PM
    date_str = date_str.replace('.', '')             # remove dots in month abbreviation

    # Step 3: Parse
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT


In [82]:
df["call_datetime"] = df["date"].apply(parse_clean_date)

C:\Users\ssmyt\AppData\Local\Temp\ipykernel_3604\2838361846.py:53: UserWarning: Parsing dates in Sept %d, %Y, %m:%M %p format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  return pd.to_datetime(date_str)


In [83]:
df["call_datetime"].isna().sum()

5

In [87]:
df[df["call_datetime"].isna()]["date"]

416                                                       
6040     [Ethan Allen Interiors Inc  (ETD -0.17%)Q2 201...
10457    [Moog Inc. (MOG.A 0.77%) (MOG.B)Q2 2021 Earnin...
10665    [Rush Enterprises Inc (RUSHA -1.94%) (RUSHB -3...
10866    [TTEC Holdings, Inc. (TTEC -0.52%)TTEC Holding...
Name: date, dtype: object

In [89]:
df.iloc[6040]['date']

['Ethan Allen Interiors Inc\xa0 (ETD -0.17%)Q2\xa02019 ',
 'Jan. 28, 2019, 5:00 p.m. ET']

In [ ]:
df